In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style()

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os, sys, time
import cPickle as pickle
sys.path.append('../EXP/')
import ClevelandMcGill as C
reload(C)

<module 'ClevelandMcGill' from '../EXP/ClevelandMcGill/__init__.pyc'>

In [19]:
N = 100000
images = np.zeros((N,100,100), dtype=np.float32)
labels = np.zeros((N), dtype=np.float32)

for n in range(N):
  
  sparse, image, label, parameters = C.Figure1.angle([True, True])
  
  noise = np.random.random_sample((100,100))
  noise[image==0] = 0.
  
  images[n] = noise
  labels[n] = label
  

In [25]:
X = images.astype(np.float32) - .5
y = (labels-np.min(labels))/(np.max(labels)-np.min(labels))
print 'memory usage', (X.nbytes + y.nbytes) / 1000000., 'MB'

memory usage 4000.4 MB


In [26]:
X3D = np.stack((X,)*3, -1)
print 'memory usage', (X3D.nbytes + y.nbytes) / 1000000., 'MB'

memory usage 12000.4 MB


In [27]:
import keras.applications

VGG19 = keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=(100,100,3))

t0 = time.time()
features = VGG19.predict(X3D, verbose=True)
print 'VGG19 features done after', time.time()-t0

Using TensorFlow backend.


100000/100000 [==============================] - 277s 3ms/step
VGG19 features done after 276.619537115


In [28]:
oshape = features.shape
print 'features shape', oshape
print 'memory usage', (features.nbytes + y.nbytes) / 1000000., 'MB'

features shape (100000, 3, 3, 512)
memory usage 1843.6 MB


In [29]:
from keras import models
from keras import layers
from keras import optimizers
import keras.applications
from keras import backend as K
from keras.utils.np_utils import to_categorical

MLP = models.Sequential()

MLP.add(layers.Dense(256, activation='relu', input_dim=oshape[1]*oshape[2]*oshape[3]))
MLP.add(layers.Dropout(0.5))
MLP.add(layers.Dense(1, activation='linear')) # REGRESSION

sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
MLP.compile(loss='mean_squared_error', optimizer=sgd, metrics=['mse', 'mae']) # MSE for regression

In [30]:
t0 = time.time()
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')]

history = MLP.fit(features[0:60000].reshape(60000, oshape[1]*oshape[2]*oshape[3]), \
                  y[0:60000], \
                  epochs=1000, \
                  batch_size=32, \
                  validation_split=0.25,
                  callbacks=callbacks,
                  verbose=True)
print 'Fitting done', time.time()-t0

Train on 45000 samples, validate on 15000 samples
Epoch 1/1000
45000/45000 [==============================] - 7s 153us/step - loss: 0.0780 - mean_squared_error: 0.0780 - mean_absolute_error: 0.2178 - val_loss: 0.0432 - val_mean_squared_error: 0.0432 - val_mean_absolute_error: 0.1712
Epoch 2/1000
45000/45000 [==============================] - 7s 147us/step - loss: 0.0482 - mean_squared_error: 0.0482 - mean_absolute_error: 0.1775 - val_loss: 0.0360 - val_mean_squared_error: 0.0360 - val_mean_absolute_error: 0.1537
Epoch 3/1000
45000/45000 [==============================] - 7s 147us/step - loss: 0.0423 - mean_squared_error: 0.0423 - mean_absolute_error: 0.1649 - val_loss: 0.0320 - val_mean_squared_error: 0.0320 - val_mean_absolute_error: 0.1430
Epoch 4/1000
45000/45000 [==============================] - 7s 146us/step - loss: 0.0386 - mean_squared_error: 0.0386 - mean_absolute_error: 0.1567 - val_loss: 0.0294 - val_mean_squared_error: 0.0294 - val_mean_absolute_error: 0.1358
Epoch 5/1000
4

Epoch 36/1000
45000/45000 [==============================] - 7s 147us/step - loss: 0.0185 - mean_squared_error: 0.0185 - mean_absolute_error: 0.1063 - val_loss: 0.0139 - val_mean_squared_error: 0.0139 - val_mean_absolute_error: 0.0912
Epoch 37/1000
45000/45000 [==============================] - 7s 147us/step - loss: 0.0184 - mean_squared_error: 0.0184 - mean_absolute_error: 0.1060 - val_loss: 0.0137 - val_mean_squared_error: 0.0137 - val_mean_absolute_error: 0.0909
Epoch 38/1000
45000/45000 [==============================] - 7s 146us/step - loss: 0.0182 - mean_squared_error: 0.0182 - mean_absolute_error: 0.1052 - val_loss: 0.0136 - val_mean_squared_error: 0.0136 - val_mean_absolute_error: 0.0906
Epoch 39/1000
45000/45000 [==============================] - 7s 147us/step - loss: 0.0179 - mean_squared_error: 0.0179 - mean_absolute_error: 0.1045 - val_loss: 0.0130 - val_mean_squared_error: 0.0130 - val_mean_absolute_error: 0.0884
Epoch 40/1000
45000/45000 [==============================] -

Epoch 71/1000
45000/45000 [==============================] - 7s 145us/step - loss: 0.0145 - mean_squared_error: 0.0145 - mean_absolute_error: 0.0937 - val_loss: 0.0104 - val_mean_squared_error: 0.0104 - val_mean_absolute_error: 0.0789
Epoch 72/1000
45000/45000 [==============================] - 7s 146us/step - loss: 0.0145 - mean_squared_error: 0.0145 - mean_absolute_error: 0.0936 - val_loss: 0.0106 - val_mean_squared_error: 0.0106 - val_mean_absolute_error: 0.0798
Epoch 73/1000
45000/45000 [==============================] - 7s 147us/step - loss: 0.0144 - mean_squared_error: 0.0144 - mean_absolute_error: 0.0934 - val_loss: 0.0108 - val_mean_squared_error: 0.0108 - val_mean_absolute_error: 0.0806
Epoch 74/1000
45000/45000 [==============================] - 7s 145us/step - loss: 0.0144 - mean_squared_error: 0.0144 - mean_absolute_error: 0.0932 - val_loss: 0.0099 - val_mean_squared_error: 0.0099 - val_mean_absolute_error: 0.0771
Epoch 75/1000
45000/45000 [==============================] -

Epoch 106/1000
45000/45000 [==============================] - 7s 147us/step - loss: 0.0127 - mean_squared_error: 0.0127 - mean_absolute_error: 0.0872 - val_loss: 0.0086 - val_mean_squared_error: 0.0086 - val_mean_absolute_error: 0.0711
Epoch 107/1000
45000/45000 [==============================] - 7s 147us/step - loss: 0.0126 - mean_squared_error: 0.0126 - mean_absolute_error: 0.0873 - val_loss: 0.0085 - val_mean_squared_error: 0.0085 - val_mean_absolute_error: 0.0705
Epoch 108/1000
45000/45000 [==============================] - 7s 145us/step - loss: 0.0125 - mean_squared_error: 0.0125 - mean_absolute_error: 0.0870 - val_loss: 0.0087 - val_mean_squared_error: 0.0087 - val_mean_absolute_error: 0.0726
Epoch 109/1000
45000/45000 [==============================] - 7s 146us/step - loss: 0.0126 - mean_squared_error: 0.0126 - mean_absolute_error: 0.0870 - val_loss: 0.0083 - val_mean_squared_error: 0.0083 - val_mean_absolute_error: 0.0701
Epoch 110/1000
45000/45000 [============================

Epoch 141/1000
45000/45000 [==============================] - 7s 146us/step - loss: 0.0114 - mean_squared_error: 0.0114 - mean_absolute_error: 0.0828 - val_loss: 0.0075 - val_mean_squared_error: 0.0075 - val_mean_absolute_error: 0.0668
Epoch 142/1000
45000/45000 [==============================] - 7s 146us/step - loss: 0.0115 - mean_squared_error: 0.0115 - mean_absolute_error: 0.0830 - val_loss: 0.0079 - val_mean_squared_error: 0.0079 - val_mean_absolute_error: 0.0691
Epoch 143/1000
45000/45000 [==============================] - 7s 145us/step - loss: 0.0115 - mean_squared_error: 0.0115 - mean_absolute_error: 0.0829 - val_loss: 0.0082 - val_mean_squared_error: 0.0082 - val_mean_absolute_error: 0.0692
Epoch 144/1000
45000/45000 [==============================] - 7s 145us/step - loss: 0.0114 - mean_squared_error: 0.0114 - mean_absolute_error: 0.0827 - val_loss: 0.0073 - val_mean_squared_error: 0.0073 - val_mean_absolute_error: 0.0655
Epoch 145/1000
45000/45000 [============================

Epoch 176/1000
45000/45000 [==============================] - 7s 145us/step - loss: 0.0107 - mean_squared_error: 0.0107 - mean_absolute_error: 0.0800 - val_loss: 0.0067 - val_mean_squared_error: 0.0067 - val_mean_absolute_error: 0.0627
Epoch 177/1000
45000/45000 [==============================] - 7s 145us/step - loss: 0.0108 - mean_squared_error: 0.0108 - mean_absolute_error: 0.0803 - val_loss: 0.0068 - val_mean_squared_error: 0.0068 - val_mean_absolute_error: 0.0632
Epoch 178/1000
45000/45000 [==============================] - 7s 145us/step - loss: 0.0105 - mean_squared_error: 0.0105 - mean_absolute_error: 0.0794 - val_loss: 0.0067 - val_mean_squared_error: 0.0067 - val_mean_absolute_error: 0.0628
Epoch 179/1000
45000/45000 [==============================] - 6s 144us/step - loss: 0.0107 - mean_squared_error: 0.0107 - mean_absolute_error: 0.0800 - val_loss: 0.0068 - val_mean_squared_error: 0.0068 - val_mean_absolute_error: 0.0638
Epoch 180/1000
45000/45000 [============================

In [31]:
y_pred = MLP.predict(features[60000:].reshape(40000, oshape[1]*oshape[2]*oshape[3]))

In [32]:
import sklearn.metrics
y_test = y[60000:]
np.log2(sklearn.metrics.mean_absolute_error(y_pred*100, y_test*100)+.125)

2.7246805861536854

In [33]:
y_pred

array([[ 0.69356406],
       [ 0.85484779],
       [-0.0266692 ],
       ..., 
       [ 0.42910156],
       [ 0.87494051],
       [ 0.2531321 ]], dtype=float32)

In [34]:
y_test

array([ 0.93258429,  0.83146065,  0.07865169, ...,  0.34831461,
        0.89887643,  0.24719101], dtype=float32)